In [246]:
from parser import public_construct, public_match

In [247]:
from utils import SocketReader

In [248]:
try:
    sock.close()
except:
    pass

sock = SocketReader("8.8.8.8", 53, 'udp')

sock.connect()

In [249]:
dns_header_spec = """
16@id
-1@qr-4@opcode-1@aa-1@tc-1@rd-1@ra-3@z-4@rcode
-16@qcount
-16@ancount
-16@nscount
-16@arcount
"""

hostname_spec = """
~\0@question(
    ...8@parts()
)
"""

dns_question_spec = hostname_spec + """
-16@type
-16@class
"""


dns_answer_spec = dns_question_spec + """
-32@ttl
-16@answer()
"""

compressed_dns_answer_spec = """
2@ones
-14@pointer
-16@type
-16@class
-32@ttl
-16@answer()
"""

In [250]:
dns_header = {
    'id': 0x911f,
    'qr': 0b0,
    'opcode': 0b0000,
    'aa': 0b0,
    'tc': 0b0,
    'rd': 0b1,
    'ra': 0b0,
    'z': 0b000,
    'rcode': 0b0000,
    'qcount': 1,
    'ancount': 0,
    'nscount': 0,
    'arcount': 0
}

dns_question = {
    'question': {
        'parts': [
            "www", "google", "com"
        ],
    },
    'type': 1,
    'class': 1
}

In [251]:
req = b''
header_data = public_construct(dns_header_spec, dns_header)
req += header_data

question_data = public_construct(dns_question_spec, dns_question)
req += question_data
print(req)

b'\x91\x1f\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x03www\x06google\x03com\x00\x00\x01\x00\x01'


In [252]:
sock.send(req)

In [253]:
resp = sock.read_once()

In [254]:
res, rem = public_match(dns_header_spec, resp)
res

{'id': 0x911f,
 'qr': 0b1,
 'opcode': 0b0000,
 'aa': 0b0,
 'tc': 0b0,
 'rd': 0b1,
 'ra': 0b1,
 'z': 0b000,
 'rcode': 0b0000,
 'qcount': 0x0001,
 'ancount': 0x0001,
 'nscount': 0x0000,
 'arcount': 0x0000}

In [255]:
questions = []
for i in range(res['qcount'].int()):
    question, rem = public_match(dns_question_spec, rem)
    questions.append(question)

questions

[{'question': {'parts': ["www", "google", "com"]},
  'type': 0x0001,
  'class': 0x0001}]

In [256]:
print(rem)
answers = []
for i in range(res['ancount'].int()):
    print(rem[0])
    if rem[0] & 0b11<<6 == 0b11<<6:
        answer, rem = public_match(compressed_dns_answer_spec, rem)
        answers.append(answer)
    else:
        answer, rem = public_match(dns_answer_spec, rem)
        answers.append(answer)

answers

b'\xc0\x0c\x00\x01\x00\x01\x00\x00\x00I\x00\x04\xac\xd9\x17\xc4'
192


[{'ones': 0b11,
  'pointer': 0b00000000001100,
  'type': 0x0001,
  'class': 0x0001,
  'ttl': 0x00000049,
  'answer': 0xacd917c4}]

In [257]:
for answer in answers:
    if answer['type'].int() == 1 and answer['class'].int() == 1:
        ip_bytes = answer['answer'].bytes()
        ip = '%d.%d.%d.%d'%tuple([int(s) for s in ip_bytes])
        if 'question' in answer:
            for_ = '.'.join(x.str() for x in answer['question']['parts'])
        else:
            tmp, _ = public_match(hostname_spec, resp[answers[0]['pointer'].int():])
            for_ = '.'.join(x.str() for x in tmp['question']['parts'])
        print("found ip", ip, "for", for_)

found ip 172.217.23.196 for www.google.com


In [258]:
sock.close()